In [1]:
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages

In [2]:
import requests
import pandas as pd 
import numpy as np 
import random 
import geopy.geocoders
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import folium 
from IPython.display import Image 
from IPython.core.display import HTML
import certifi
import ssl
from geopy.geocoders import Nominatim

In [3]:
dic = {'Neighborhood': ['Carignano','San Quirico','Borzoli','Oregina', 'Castelletto',  'Lagaccio',  'Maddalena',  'Prè',  'Molo', 'San Teodoro',  
       'Sampierdarena', 'Marassi',  'San Fruttuoso', 'Staglieno',  'Molassana',  'Struppa', 'Rivarolo',  'Bolzaneto',
       'Pontedecimo', 'Sestri Ponente',  'Cornigliano', 'Voltri',  'Pra',  'Pegli', 'Foce',  'Albaro',  'San Martino',
       'Sturla',  'Quarto',  'Quinto',  'Nervi'], 'Population': [7008,4002,4951,12659,28857,12906,5825,7839,9635,23049,44346,39536,34444,21684,24868,10011,34165,15334,8173,44798,15481,13096,21766,26310,15722,28963,15130,8278,41197,8439,10739], 
       'Area':[0.591,4.05,3.3,1.85,1.99,0.89,0.27,0.45,0.35,1.78,3.07,5.82,1.4,6.74,16.61,19.13,12.30,9.98,3.14,3.55,2.43,34.84,4.32,3.24,0.24,0.93,0.98,0.78,1,2.89,15.77],
      'Income':[37913,20001,20139,21278,34716,19614,27684,22460,24253,23407,20082,20439,21459,20439,20038,20059,19748,18359,20358,20995,
               17367,19330,19165,26265,33687,40059,26339,27459,27042,29064,30273]}
df=pd.DataFrame(data=dic)
df['Latitude']= [None]*df.shape[0]
df['Longitude']=[None]*df.shape[0]
df.head()

,Neighborhood,Population,Area,Income,Latitude,Longitude
0,Carignano,7008,0.591,37913,None,None
1,San Quirico,4002,4.050,20001,None,None
2,Borzoli,4951,3.300,20139,None,None
3,Oregina,12659,1.850,21278,None,None
4,Castelletto,28857,1.990,34716,None,None


In [4]:
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
i=0
for element in df['Neighborhood']:
    address = element + ' quartiere,Genova città,Italia'
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    df.loc[i,'Latitude'] = location.latitude
    df.loc[i,'Longitude'] = location.longitude
    i=i+1
df['Latitude']=df['Latitude'].apply(float)
df['Longitude']=df['Longitude'].apply(float)
df.dtypes

Neighborhood     object
Population        int64
Area            float64
Income            int64
Latitude        float64
Longitude       float64
dtype: object

In [5]:

genovalat = 44.409903
genovalong = 8.918240
genova_map = folium.Map(location=[genovalat, genovalong], zoom_start=12)

for label, lat, lng in zip(df['Neighborhood'],df['Latitude'],df['Longitude']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=1
    ).add_to(genova_map)

genova_map

In [6]:
import math
def radius(area,factor1,factor2):
    df['Radius'] = [None]*df.shape[0]
    i=0
    for x in area:
        if x < 1:
            df.loc[i,'Radius'] = (math.sqrt((x*1000000)/3.1415926))/factor1
        else:
            df.loc[i,'Radius'] = (math.sqrt((x*1000000)/3.1415926))/factor2
        i = i+1
        
radius(df['Area'],0.8,1.5) #We will arbitrarily choose the two factors to try to minimize the overlap
df['Radius'] = df['Radius'].apply(int)

In [7]:
genovalat = 44.409903
genovalong = 8.918240
genova_map = folium.Map(location=[genovalat, genovalong], zoom_start=12)

for label, lat, lng, rad in zip(df['Neighborhood'],df['Latitude'],df['Longitude'],df['Radius']):
    folium.features.Circle(
        [lat, lng],
        radius=rad,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.2
    ).add_to(genova_map)

for label, lat, lng in zip(df['Neighborhood'],df['Latitude'],df['Longitude']):
    folium.features.Marker(
        [lat, lng],
        #radius=5,
        #color='red',
        popup=label,
        #fill = True,
        #fill_color='red',
        #fill_opacity=1
    ).add_to(genova_map)
    
genova_map

In [8]:
CLIENT_ID = 'CUKF3VRXRNKMVUC0ZQXLLJPN3R1TIQB5YX5TFBDMF1PKEVPR'
CLIENT_SECRET = '3XGOEQ01BTLZQ35V5A4NCUW1FFNJQTH1QRA0C2ULCUJXND22'
VERSION = '20190913'
LIMIT = 100

In [9]:
dic = {'Hotel':[],'Ristorante':[],'Bar':[],'Pub':[],'Caffè':[]}
places = pd.DataFrame(data=dic)
pl = places.columns

In [10]:
def info(place,lat,long,rad,datf):
    res = []
    for y,z,k in zip(lat,long,rad):
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, y,z, VERSION, place, k, 100)
        results = requests.get(url).json()
        res.append((results['response']['totalResults']))
    datf[place] = res

In [11]:
for pla in pl:
    info(pla,df['Latitude'],df['Longitude'],df['Radius'],places)

In [12]:
df1 = pd.concat([df,places],join='inner',axis=1)

In [14]:
df1.head()

,Neighborhood,Population,Area,Income,Latitude,Longitude,Radius,Hotel,Ristorante,Bar,Pub,Caffè
0,Carignano,7008,0.591,37913,44.401700,8.937117,542,6,10,18,1,4
1,San Quirico,4002,4.050,20001,44.482624,8.901147,756,0,0,0,0,0
2,Borzoli,4951,3.300,20139,44.434394,8.867475,683,0,0,0,0,0
3,Oregina,12659,1.850,21278,44.422988,8.925229,511,0,0,0,0,0
4,Castelletto,28857,1.990,34716,44.415524,8.940414,530,1,2,2,0,0


In [15]:
import json
with open('mappa.geojson') as json_data:
    genova_data = json.load(json_data)

FileNotFoundError: [Errno 2] No such file or directory: 'mappa.geojson'